<a href="https://colab.research.google.com/github/arjunmalik11/AI-Agents-with-Autogen/blob/main/Autogen_Reflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
OPENAI_API_KEY = "secret-key"
llm_config = {
    "model" : "gpt-4",
    "api_key" : OPENAI_API_KEY,
 }

In [3]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


Writer Agent

In [4]:
import autogen

task = """
Write a concise but engaging blogpost about
Autogen. Make sure the blogpost is
within 100 words.
"""

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)


reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])
reply

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[autogen.oai.client: 09-11 16:28:38] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


"Title: Revolutionizing Development with Autogen\n\nImagine a tool that automates tedious scripting tasks, saves your time and boosts your productivity. Yes, it's Autogen. A vital player in the software development environment, Autogen simplifies the code generation process by swiftly transforming specifications into reusable program code. This game-changer tool propels efficiency, reduces human error, and remarkably eases the software maintenance process. With its dynamic template-driven system, Autogen relieves developers from daunting, repetitive tasks, allowing them to delve deeper into creative problem-solving. Get on board with Autogen, embrace technological innovation, and experience the new era of software development!"

Adding Reflection to Writer Agent

In [5]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

[autogen.oai.client: 09-11 16:30:04] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Critic (to Writer):


Write a concise but engaging blogpost about
Autogen. Make sure the blogpost is
within 100 words.


--------------------------------------------------------------------------------
Writer (to Critic):

Title: Autogen: A Game Changer in Code Generation

Autogen is revolutionizing the tech industry with its cutting-edge code generation capabilities. This versatile tool simplifies programming by automating a variety of tasks, saving time while mitigating the risk of human error. Whether it's configuration file templates or developing complex software, Autogen is your reliable ally. Embrace the power of Autogen for a smoother programming journey and upgrade your efficiency. Welcome to the future of coding!

--------------------------------------------------------------------------------
Critic (to Writer):

The blogpost provides a clear and exciting introduction to Autogen. It rightly highlights the benefits of the tool and its impact on the programming world. However,

Creating Nested Chats for better reflection

In [6]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (2 bullets max), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (2 bullets max), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message = "You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content."
)

[autogen.oai.client: 09-11 16:33:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-11 16:33:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-11 16:33:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-11 16:33:03] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Combining the complete orchestration

In [7]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]

critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


Write a concise but engaging blogpost about
Autogen. Make sure the blogpost is
within 100 words.


--------------------------------------------------------------------------------
Writer (to Critic):

Title: Autogen: A Game Changer in Code Generation

Autogen is revolutionizing the tech industry with its cutting-edge code generation capabilities. This versatile tool simplifies programming by automating a variety of tasks, saving time while mitigating the risk of human error. Whether it's configuration file templates or developing complex software, Autogen is your reliable ally. Embrace the power of Autogen for a smoother programming journey and upgrade your efficiency. Welcome to the future of coding!

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

******************************************************************************

/usr/local/lib/python3.10/dist-packages/autogen/agentchat/conversable_agent.py:1264: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to Critic):

As a legal reviewer, I suggest the following:

1. Avoid mentioning that Autogen is revolutionizing the tech industry without concrete evidence to support such a claim, as it may be misinterpreted as false advertising.

2. Refrain from guaranteeing an upgrade in efficiency or a smoother programming journey without clarifying conditions or circumstances, as it could potentially be viewed as a definite warranty or promise.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Critic (to Ethics Reviewer):

Review the following content. 
            

 Title: Autogen: A Game Changer in Code Generation

Autogen is revolutionizing the tech industry with its cutting-edge code generation capabilities. This versatile tool simplifies programming by automat